¿Para qué sirve el modelo?
El modelo DETR (DEtection TRansformer) con la arquitectura ResNet-101 se utiliza para la detección de objetos en imágenes. Este proceso consiste en identificar automáticamente los diferentes objetos que aparecen en una imagen y determinar su ubicación exacta dentro de ella. Es decir, el modelo no solo reconoce qué objetos están presentes, sino que también traza un cuadro delimitador (bounding box) alrededor de cada uno, especificando sus coordenadas.

Esta tecnología es esencial en numerosos campos de la visión por computadora, donde es necesario procesar grandes cantidades de imágenes para obtener información relevante sobre los objetos que contienen. Por ejemplo, en la seguridad y vigilancia, puede ser utilizado para detectar personas o vehículos en las grabaciones de cámaras de seguridad. En el ámbito médico, puede ayudar a los radiólogos a identificar y localizar anomalías en imágenes médicas como radiografías o resonancias magnéticas. Asimismo, en la conducción autónoma, es clave para que los vehículos reconozcan y reaccionen a su entorno, detectando otros coches, peatones, señales de tráfico, y obstáculos en la carretera.

¿Quién o qué empresa ha entrenado el modelo?
El modelo fue entrenado por Facebook AI Research (FAIR), ahora conocido como Meta AI, anteriormente Facebook. FAIR es un equipo de científicos especializados en inteligencia artificial que trabajan para Meta, con el objetivo de desarrollar tecnologías que permitan a las máquinas "ver" y "comprender" el entorno de manera similar a como lo hacemos los seres humanos. Para crear DETR, estos expertos integraron diversas técnicas de inteligencia artificial, algunas previamente utilizadas en otros campos como el análisis de texto, y las adaptaron para la detección de objetos en imágenes. Su enfoque es abierto, compartiendo sus investigaciones y liberando código y herramientas, como PyTorch, como PyTorch, para la comunidad.

¿Con qué datos ha sido entrenado el modelo?
El modelo ha sido entrenado utilizando el conjunto de datos COCO 2017 (Common Objects in Context), que contiene aproximadamente 118,000 imágenes etiquetadas. Cada imagen viene acompañada de información detallada sobre los objetos que aparecen en ella. Por ejemplo, si en una imagen hay un perro, un coche y una persona, cada uno de estos objetos estará identificado con su respectiva etiqueta (como "perro", "coche" o "persona"), y se indican las coordenadas exactas que definen un cuadro delimitador (bounding box) alrededor de cada objeto. El conjunto de datos incluye más de 80 categorías de objetos diferentes, desde animales y vehículos hasta utensilios de cocina y señales de tráfico, asegurando que el modelo pueda aprender a detectar una amplia variedad de objetos en diversos contextos y situaciones.  A medida que procesa miles de imágenes, ajusta sus parámetros internos para mejorar su capacidad de identificar correctamente los objetos y predecir sus ubicaciones en nuevas imágenes, lo que es esencial para su precisión y eficacia en aplicaciones del mundo real.



¿Qué posible salida puede devolver el modelo?
El modelo DETR (DEtection TRansformer) puede generar múltiples salidas al analizar una imagen. 
- Clases de los objetos detectados: El modelo identifica los tipos de objetos que aparecen en la imagen. Por ejemplo, podría reconocer un "gato", un "coche" o una "persona". Cada objeto detectado se asocia con una etiqueta de clase que indica su naturaleza.
- Puntuaciones de confianza: Para cada objeto detectado, el modelo proporciona una puntuación que refleja su grado de certeza sobre la detección. Esta puntuación de confianza varía entre 0 y 1, donde un valor cercano a 1 indica que el modelo está muy seguro de haber identificado correctamente el objeto. Por ejemplo, podría afirmar que está un 99% seguro de que un objeto es un "gato".
- Cuadros delimitadores (bounding boxes):
El modelo también devuelve las coordenadas que describen un cuadro alrededor de cada objeto detectado en la imagen. Estas coordenadas definen un rectángulo (el cuadro delimitador) que indica con precisión la ubicación del objeto. Este rectángulo se representa mediante cuatro números que especifican los bordes del cuadro: las posiciones de los lados izquierdo, superior, derecho e inferior.

Por ejemplo, si el modelo analiza una imagen que muestra una cocina en la que hay una mesa con una manzana, un plato, y una jarra de agua. El modelo podría devolver resultados como los siguientes:
- Objeto 1: Manzana, con una confianza del 98.5%, ubicado en el cuadro delimitador [150.32, 120.45, 200.67, 180.89].
- Objeto 2: Plato, con una confianza del 97.2%, ubicado en el cuadro delimitador [220.12, 140.67, 300.45, 220.93].
- Objeto 3: Jarra de agua, con una confianza del 95.0%, ubicada en el cuadro delimitador [310.50, 80.30, 370.15, 180.75].

Esta información es vital en aplicaciones como la automatización del hogar, donde se necesita reconocer y gestionar elementos en el entorno.

In [1]:
#Se importan las librerías necesarias
import torch
from PIL import Image
import requests
from transformers import DetrImageProcessor, DetrForObjectDetection

# Se verifica uso de GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

ModuleNotFoundError: No module named 'torch'

In [2]:
#Se carga el modelo y el procesador
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

# you can specify the revision tag if you don't want the timm dependency
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-101", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-101", revision="no_timm")

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# convert outputs (bounding boxes and class logits) to COCO API
# let's only keep detections with score > 0.9
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
    )

Detected cat with confidence 0.998 at location [344.06, 24.85, 640.34, 373.74]
Detected remote with confidence 0.997 at location [328.13, 75.93, 372.81, 187.66]
Detected remote with confidence 0.997 at location [39.34, 70.13, 175.56, 118.78]
Detected cat with confidence 0.998 at location [15.36, 51.75, 316.89, 471.16]
Detected couch with confidence 0.995 at location [-0.19, 0.71, 639.73, 474.17]


In [3]:
#Se seleccionan imagenes al azar para probar el modelo
url = "http://images.cocodataset.org/val2017/000000281759.jpg"
image = Image.open(requests.get(url, stream=True).raw)

#Se procesa la imagen
inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

#Se convierten las salidas a la API de COCO
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

#Se imprimen los resultados
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
    )

Detected person with confidence 0.999 at location [381.7, 136.93, 465.25, 417.76]
Detected umbrella with confidence 0.999 at location [194.92, 80.89, 327.29, 211.31]
Detected person with confidence 0.999 at location [207.91, 128.04, 286.73, 406.64]
Detected umbrella with confidence 0.998 at location [54.14, 72.38, 207.28, 207.45]
Detected person with confidence 0.995 at location [298.2, 143.06, 378.06, 413.69]
Detected umbrella with confidence 0.998 at location [346.83, 99.26, 478.11, 222.89]
Detected handbag with confidence 0.979 at location [264.1, 187.96, 284.92, 233.42]
Detected umbrella with confidence 0.99 at location [464.02, 90.11, 603.31, 216.48]
Detected person with confidence 0.999 at location [454.08, 140.33, 534.55, 421.04]
Detected umbrella with confidence 0.994 at location [258.02, 227.61, 426.74, 347.87]
Detected car with confidence 0.99 at location [33.45, 300.07, 105.07, 321.56]
Detected person with confidence 1.0 at location [96.12, 95.84, 197.52, 403.26]
